In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Tue Apr 25 23:07:51 2023

@author: ddzyba
"""

#!/usr/bin/env python
# coding: utf-8

#!pip install flask-ngrok


import pandas as pd
import statsmodels.api as sm
import numpy as np
from tensorflow import keras
import os
from sklearn.preprocessing import StandardScaler

from scipy import stats

import pickle



os.environ['KMP_DUPLICATE_LIB_OK']='True'

NN3 = keras.saving.load_model('model\\NN3')
#NN3 = keras.models.load_model('model\\NN3')


#from google.colab.output import eval_js
#print(eval_js("google.colab.kernel.proxyPort(5000)"))

from flask import Flask, render_template, request
app = Flask(__name__)
#app = Flask(__name__, template_folder=r'D:\Programming\Python_files\Flask_projects\flask 2304\flask 2304\KFH2304\KFH\templates')




# Функция денормализации по yeojohnson
#  inverse of the Yeo-Johnson transformation
def yeojohnson_inverse(X_trans, lambda_):
  '''
  if X >= 0 and lambda_ == 0:
    X = np.exp(X_trans) - 1
  elif X >= 0 and lambda_ != 0:
      X = (X_trans * lambda_ + 1) ** (1 / lambda_) - 1
  elif X < 0 and lambda_ != 2:
      X = 1 - (-(2 - lambda_) * X_trans + 1) ** (1 / (2 - lambda_))
  elif X < 0 and lambda_ == 2:
      X = 1 - np.exp(-X_trans)
  '''

  if lambda_ == 0:
    X = np.exp(X_trans) - 1
  elif lambda_ != 0:
    X = (X_trans * lambda_ + 1) ** (1 / lambda_) - 1  

  return X








def check_params(x1, x2):
    message = f"X1 = {x1}, X2 = {x2}"
    return message

def print_params_for_NN(x1, x2, x3):
    if (x1 == "" or x2 == "" or x3 == ""):
        message = "ОШИБКА! Вы не ввели параметры."
    elif(float(x1) >= 0 and float(x2) >=0 and float(x3) >=0):
        message = f"Работники, чел.: {x1}\nНаличие тракторов, шт.: {x2}\nОбщая площадь земли, га: {x3}"
    else:
        message = "ОШИБКА! Введенные значения должны быть больше или равны 0."
    return message

def calculate_NN(x1, x2, x3):
    X_use = pd.DataFrame([[x1,x2,x3]],columns=['Работники, чел.','Наличие тракторов, шт.','Общая площадь земли, га'])
    X_use = np.array([[x1, x2, x3]])
    
    dummy_column = np.zeros(shape = (X_use.shape[0], 1))
    cols_indexes = [0, 3, 5] # Индексы фиктивных столбцов (для Standard Scaler)
    X_use_6cols = np.insert(X_use, 0, dummy_column.T, axis = 1)
    X_use_6cols = np.insert(X_use_6cols, 3, dummy_column.T, axis = 1)
    X_use_6cols = np.insert(X_use_6cols, 5, dummy_column.T, axis = 1)
    
    # Нормализация по yeojohnson
    lamda_list = [0.013865117288407803,
                  -1.3939833064592597,
                  -0.0542790101104967,
                  0,
                  -0.030491839284671637,
                  0] # Список значений lambda для каждого столбца (у фиктивных = 0)
    
    X_use_6cols_normalized = X_use_6cols.copy()
    
    #power_tranformer = PowerTransformer(method = 'yeo-johnson', standardize = False)
    
    #power_tranformer
    X_use_6cols_normalized[:, 1] = stats.yeojohnson(X_use_6cols_normalized[:, 1], lmbda = lamda_list[1]) 
    X_use_6cols_normalized[:, 2] = stats.yeojohnson(X_use_6cols_normalized[:, 2], lmbda = lamda_list[2]) 
    X_use_6cols_normalized[:, 4] = stats.yeojohnson(X_use_6cols_normalized[:, 4], lmbda = lamda_list[4]) 
    
    # Загрузка standardscaler
    #with open('/content/drive/My Drive/Colab Notebooks/scaler.pkl','wb') as f:
        #pickle.dump(standadscaler, f)
    with open('scaler\\scaler.pkl','rb') as f:
        standadscaler = pickle.load(f)
    
    # Стандартизация
    X_use_6cols_standard = standadscaler.transform(X_use_6cols_normalized)
    
    # Удаление фиктивных столбцов
    X_use_standard = np.delete(X_use_6cols_standard, 5, axis = 1)
    X_use_standard = np.delete(X_use_standard, 3, axis = 1)
    X_use_standard = np.delete(X_use_standard, 0, axis = 1)
        
    model_loaded = NN3
    y_pred_use = model_loaded.predict(X_use)
    print('y_pred_use =', y_pred_use)
    
    
    y_pred_use_standard = model_loaded.predict(X_use_standard) # Прогноз по нормализованным и стандартизированным
    print('y_pred_use_standard =', y_pred_use_standard)
    
    y_pred_use_standard_6cols = np.zeros(shape = (X_use.shape[0], 6))
    y_pred_use_standard_6cols[:, 0] = y_pred_use_standard[:, 0]
    
    # Дестандартизация Y_pred
    y_pred_use_standard_inv = standadscaler.inverse_transform(y_pred_use_standard_6cols)[:, 0]
    print('y_pred_use_standard_inv =', y_pred_use_standard_inv)
    
    '''
    # Денормализация Y_pred (scikit-learn)
    y_pred_use_normalized_inv_1 = y_pred_use_standard_inv.copy()
    y_pred_use_normalized_inv_1 = power_transformer_Y.inverse_transform(X = y_pred_use_normalized_inv_1.reshape(-1, 1))
    print('y_pred_use_normalized_inv_1 =', y_pred_use_normalized_inv_1)
    '''
    
    # Денормализация Y_pred (2 способ, собственная функция)
    y_pred_use_normalized_inv_2 = y_pred_use_standard_inv.copy()
    y_pred_use_normalized_inv_2 = yeojohnson_inverse(X_trans = y_pred_use_normalized_inv_2, lambda_ = lamda_list[0])
    print('y_pred_use_normalized_inv =', y_pred_use_normalized_inv_2)
    
    
    
    message = ": ".join(["Доход, тыс. руб.", str(y_pred_use_normalized_inv_2[0])])
    
    
    
    
    
    # Содержание
    
    '''
    
    standardscaler = StandardScaler()
    transform_X = standardscaler.fit_transform(X)
    
    #res = NN3.predict(transform_X)
    res = 1
    
    
    # create empty table with 3 fields
    Predict_dataset_like = np.zeros(shape=(len(X), 3) )
    # put the predicted values in the right field
    Predict_dataset_like[:,0] = res #res[:,0]
    # inverse transform and then select the right field
    res_inverse = standardscaler.inverse_transform(Predict_dataset_like)[:,0]
    message = ": ".join(["Доход, тыс. руб.", str(res_inverse[0])])
    '''
    
    
    return message

@app.route("/", methods=["post", "get"])
def index():
    message = ''
    message2 = ''
    x1=0
    x2=0
    x3=0
    if request.method == "POST":
        x1 = request.form.get("x1")
        x2 = request.form.get("x2")
        x3 = request.form.get("x3")
        
        message = print_params_for_NN(x1, x2, x3) 
        if "ОШИБКА" not in message:
            message2 = calculate_NN(float(x1), float(x2), float(x3))
        
    return render_template("index.html", message=message, message2=message2, x1=x1, x2=x2, x3=x3)

'''
if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000)
    '''
app.run()


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [25/Apr/2023 23:53:04] "GET / HTTP/1.1" 200 -


1/1 [==============================] - ETA: 0s

C:\Users\DELL\anaconda3\lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator StandardScaler from version 1.2.2 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(
C:\Users\DELL\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


1/1 [==============================] - 0s 109ms/step
y_pred_use = [[500.84595]]
1/1 [==============================] - 0s 24ms/step


...

y_pred_use_standard = [[-0.22203673]]
y_pred_use_standard_inv = [8.49809951]
y_pred_use_normalized_inv = [3081.80806787]
1/1 [==============================] - 0s 24ms/step
y_pred_use = [[1193.4205]]
1/1 [==============================] - 0s 24ms/step


C:\Users\DELL\anaconda3\lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator StandardScaler from version 1.2.2 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(
C:\Users\DELL\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
127.0.0.1 - - [25/Apr/2023 23:54:46] "POST / HTTP/1.1" 200 -


y_pred_use_standard = [[1.6162785]]
y_pred_use_standard_inv = [11.21123296]
y_pred_use_normalized_inv = [33549.2363636]
1/1 [==============================] - 0s 16ms/step
y_pred_use = [[500.84595]]
1/1 [==============================] - 0s 16ms/step


C:\Users\DELL\anaconda3\lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator StandardScaler from version 1.2.2 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(
C:\Users\DELL\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
127.0.0.1 - - [25/Apr/2023 23:57:12] "POST / HTTP/1.1" 200 -


y_pred_use_standard = [[0.14929162]]
y_pred_use_standard_inv = [9.04613575]
y_pred_use_normalized_inv = [5024.12978278]
